# Tutorial 3
The model we developed in the previous tutorial classified MNIST successfully but was rather slow. Like ANNs, to maximise performance when simulating small SNNs like this on a GPU, we need to simulate multiple copies of the model at once and run them on **batches** of input images.
In this tutorial we will modify our model to do just that as well as off-loading further computation to the GPU to improve performance.

## Install PyGeNN wheel from Google Drive
Download wheel file

In [ ]:
!gdown 1-4H8-H48tEnb-9I9EtLyNfPdLQLe2JRV

and then install PyGeNN from wheel file

In [ ]:
!pip install pygenn-4.8.0-cp38-cp38-linux_x86_64.whl

Set environment variable to allow GeNN to find CUDA

In [3]:
%env CUDA_PATH=/usr/local/cuda

env: CUDA_PATH=/usr/local/cuda


## Download pre-trained weights and MNIST test data

In [4]:
!gdown 1cmNL8W0QZZtn3dPHiOQnVjGAYTk6Rhpc
!gdown 131lCXLEH6aTXnBZ9Nh4eJLSy5DQ6LKSF
!gdown 14nSMTTlFebhqkudWVpuWnNFFEQx_7lFg
!gdown 1KSLTNx--oNoV1lEGPmo0FQbUlgsmaBKS

Downloading...
From: https://drive.google.com/uc?id=1cmNL8W0QZZtn3dPHiOQnVjGAYTk6Rhpc
To: /content/weights_0_1.npy
100% 402k/402k [00:00<00:00, 66.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=131lCXLEH6aTXnBZ9Nh4eJLSy5DQ6LKSF
To: /content/weights_1_2.npy
100% 5.25k/5.25k [00:00<00:00, 7.49MB/s]
Downloading...
From: https://drive.google.com/uc?id=14nSMTTlFebhqkudWVpuWnNFFEQx_7lFg
To: /content/testing_images.npy
100% 7.84M/7.84M [00:00<00:00, 209MB/s]
Downloading...
From: https://drive.google.com/uc?id=1KSLTNx--oNoV1lEGPmo0FQbUlgsmaBKS
To: /content/testing_labels.npy
100% 10.1k/10.1k [00:00<00:00, 13.2MB/s]


## Build model
Import standard module and PyGeNN functionality as before and configure simulation parameters

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from pygenn.genn_model import (create_custom_neuron_class,
                               create_custom_current_source_class,
                               create_custom_custom_update_class,
                               create_var_ref,
                               GeNNModel)
from time import perf_counter
from tqdm import tqdm

TIMESTEP = 1.0
PRESENT_TIMESTEPS = 100
INPUT_CURRENT_SCALE = 1.0 / 100.0

As we're going to use it in a few places, we add an additional simulation parameter to define the batch size.

In [6]:
BATCH_SIZE = 128

Define the custom neuron and synapse models in exactly the same way as before

In [7]:
# Very simple integrate-and-fire neuron model
if_model = create_custom_neuron_class(
    "if_model",
    param_names=["Vthr"],
    var_name_types=[("V", "scalar"), ("SpikeCount", "unsigned int")],
    sim_code="$(V) += $(Isyn) * DT;",
    reset_code="""
    $(V) = 0.0;
    $(SpikeCount)++;
    """,
    threshold_condition_code="$(V) >= $(Vthr)")

cs_model = create_custom_current_source_class(
    "cs_model",
    var_name_types=[("magnitude", "scalar")],
    injection_code="$(injectCurrent, $(magnitude));")


As we increase the batch size of our model, the cost of resetting the spike counts and membrane voltages will increase. To counteract this, we can offload tasks like this to the GPU using a *custom update* model. These are defined using very similar syntax to neuron and synapse models but have one additional feature - variable references. These allow custom updates to be *attached* to existing neuron or synapse populations to modify their variables outside of the standard neuron and synapse updates.

In [8]:
reset_model = create_custom_custom_update_class(
    "reset",
    param_names=[],
    var_name_types=[],
    var_refs=[("V", "scalar"), ("SpikeCount", "unsigned int")],
    update_code="""
    $(V) = 0.0;
    $(SpikeCount) = 0;
    """)

Create a new model in exactly the same way as before

In [9]:
model = GeNNModel("float", "tutorial_3")
model.dT = TIMESTEP

Set the model batch size

In [10]:
model.batch_size = BATCH_SIZE

Build model, load weights and create neuron, synapse and current source populations as before

In [11]:
# Load weights
weights_0_1 = np.load("weights_0_1.npy")
weights_1_2 = np.load("weights_1_2.npy")

if_params = {"Vthr": 5.0}
if_init = {"V": 0.0, "SpikeCount":0}
neurons = [model.add_neuron_population("neuron0", weights_0_1.shape[0],
                                       if_model, if_params, if_init),
           model.add_neuron_population("neuron1", weights_0_1.shape[1],
                                       if_model, if_params, if_init),
           model.add_neuron_population("neuron2", weights_1_2.shape[1],
                                       if_model, if_params, if_init)]
model.add_synapse_population(
        "synapse_0_1", "DENSE_INDIVIDUALG", 0,
        neurons[0], neurons[1],
        "StaticPulse", {}, {"g": weights_0_1.flatten()}, {}, {},
        "DeltaCurr", {}, {})
model.add_synapse_population(
        "synapse_1_2", "DENSE_INDIVIDUALG", 0,
        neurons[1], neurons[2],
        "StaticPulse", {}, {"g": weights_1_2.flatten()}, {}, {},
        "DeltaCurr", {}, {});

current_input = model.add_current_source("current_input", cs_model,
                                         neurons[0], {}, {"magnitude": 0.0})

In [12]:
for n in neurons:
    reset_var_refs = {"V": create_var_ref(n, "V"),
                      "SpikeCount": create_var_ref(n, "SpikeCount")}
    model.add_custom_update(f"{n.name}_optimizer", "Reset", reset_model,
                            {}, {}, reset_var_refs)

In [13]:
# Build and load our model
model.build()
model.load()

testing_images = np.load("testing_images.npy")
testing_labels = np.load("testing_labels.npy")

assert testing_images.shape[1] == weights_0_1.shape[0]
assert np.max(testing_labels) == (weights_1_2.shape[1] - 1)

First of all, we determine where to split our test data to achieve our batch size and then use `np.split` to perform the splitting operation (the last batch will contain < `BATCH_SIZE` stimuli as 128 does not divide 10000 evenly)

In [14]:
batch_splits = range(BATCH_SIZE, testing_images.shape[0] + 1, BATCH_SIZE)

testing_image_batches = np.split(testing_images, batch_splits, axis=0)
testing_label_batches = np.split(testing_labels, batch_splits, axis=0)

## Simulate model
Our batched simulation loop looks very similar to the loop we defined in the previous tutorial however:
*   We now loop over *batches* of images and labels rather than individual ones
*   When we copy images into the input current view, we only copy as many images as are present in this batch to handle the remainder in the final batch
*   We specify an axis for `np.argmax` so that we get the neuron with the largest spike count in each batch



In [15]:
current_input_magnitude = current_input.vars["magnitude"].view
output_spike_count = neurons[-1].vars["SpikeCount"].view
neuron_voltages = [n.vars["V"].view for n in neurons]

# Simulate
num_correct = 0
start_time = perf_counter()
for img, lab in tqdm(zip(testing_image_batches, testing_label_batches)):
    current_input_magnitude[:img.shape[0],:] = img * INPUT_CURRENT_SCALE
    current_input.push_var_to_device("magnitude")

    # Run reset custom update
    model.custom_update("Reset")

    for t in range(PRESENT_TIMESTEPS):
        model.step_time()

    # Download spike count from last layer
    neurons[-1].pull_var_from_device("SpikeCount")

    # Find which neuron spiked most in each batch to get prediction
    predicted_lab = np.argmax(output_spike_count, axis=1)

    # Add number of  
    num_correct += np.sum(predicted_lab[:lab.shape[0]] == lab)

end_time = perf_counter()
print(f"\nAccuracy {((num_correct / float(testing_images.shape[0])) * 100.0)}%%")
print(f"Time {end_time - start_time} seconds")


79it [00:00, 104.96it/s]


Accuracy 97.54%%
Time 0.7618121939999583 seconds


And...we get a speed up of over 30x compared to the previous tutorial